# Custom Evaluation Criteria

In this example, we will see how to create a custom metric.

We will create a metric that evaluates whether a user query is decomposed into sub-queries, covering all the angles of the original query to retrieve all the necessary information.

# Creating the metric

`flow-judge` makes it easy to create custom metrics.

In [1]:
from flow_judge.metrics import CustomMetric, RubricItem
from IPython.display import Markdown, display

# Define the criteria
evaluation_criteria = """Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?"""

# Define the rubric using RubricItem's
rubric = [
    RubricItem(
        score=1,
        description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."),
    RubricItem(
        score=2,
        description="The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic."),
    RubricItem(
        score=3,
        description="The sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. They break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented in the sub-queries, allowing for a thorough and well-rounded investigation of the subject matter."),
]

# We need to define the required inputs and output for the metric
required_inputs = ["query"]
required_output = "sub_queries"

# Create the metric
sub_query_coverage = CustomMetric(
    name="sub-query-coverage",
    criteria=evaluation_criteria,
    rubric=rubric,
    required_inputs=required_inputs,
    required_output=required_output
)

As you can see, we quickly created a custom metric that will instruct the model to evaluate according to the criteria and rubric we set.

In [2]:
sub_query_coverage

CustomMetric(name='sub-query-coverage', criteria='Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?', rubric=[RubricItem(score=1, description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."), RubricItem(score=2, description='The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic.'), RubricItem(score=3, description='The sub-queries demonstrate exce

# Running evaluations with the custom metric

Now, we just need to initialize the judge with our custom metric and run the evaluations.

In [3]:
from flow_judge.flow_judge import EvalInput, FlowJudge
from flow_judge.models import Vllm #, Llamafile, Hf

# If you are running on an Ampere GPU or newer, create a model using VLLM
model = Vllm()

# Or if not running on Ampere GPU or newer, create a model using no flash attn and Hugging Face Transformers
# model = Hf(flash_attn=False)

# Or create a model using Llamafile if not running an Nvidia GPU & running a Silicon MacOS for example
# model = Llamafile()

# Initialize the judge
judge = FlowJudge(metric=sub_query_coverage, model=model)

INFO 09-24 11:16:28 awq_marlin.py:89] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 09-24 11:16:28 config.py:383] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-24 11:16:28 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decodin

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-24 11:16:30 model_runner.py:1008] Loading model weights took 2.1861 GB
INFO 09-24 11:16:31 gpu_executor.py:122] # GPU blocks: 3085, # CPU blocks: 682


In [9]:
# Sample to evaluate

query = "I placed an order for a custom-built gaming PC (order #AC-789012) two weeks ago, but the estimated delivery date has changed twice since then. Originally it was supposed to arrive yesterday, then it got pushed to next week, and now the tracking page shows 'Status: Processing' with no delivery estimate at all. I've tried calling customer service, but after waiting on hold for an hour, I was told to check the website. Can you please look into this and explain what's causing the delays, when I can realistically expect my order to arrive, and whether I'm eligible for any kind of compensation or expedited shipping given these repeated delays? I'm especially concerned because I need this computer for an upcoming gaming tournament I'm participating in next month."
sub_queries = "What is the current shipping status of my order? How can I build a PC?"

# bad decomposition
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.48s/it, est. speed input: 152.88 toks/s, output: 44.78 toks/s]


In [10]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries fail to provide sufficient breadth to cover all aspects of the main query. The main query addresses multiple issues: repeated delivery date changes, current shipping status, potential compensation, and expedited shipping options. However, the sub-queries only touch on two narrow aspects: the current shipping status and how to build a PC.

The sub-queries completely miss several critical elements of the main query:
1. The reason for repeated delivery date changes
2. The current status of the order
3. Eligibility for compensation
4. Options for expedited shipping
5. The impact on the upcoming gaming tournament

By focusing only on the current shipping status and PC building, the sub-queries significantly limit the scope of the investigation. They fail to address the customer's concerns about repeated delays, potential compensation, and the urgency of the situation due to the upcoming gaming tournament.

Using these sub-queries alone would result in a severely limited exploration of the topic, failing to address the majority of the customer's concerns and questions. The sub-queries do not provide a comprehensive breakdown of the main query's dimensions, leaving many important aspects unexplored.

__Score:__
1

In [6]:
# Good decomposition

sub_queries = """1. What is the current status of order #AC-789012, and why has it changed from the original estimated delivery date?
2. What specific factors are causing the delays in processing and shipping this custom-built gaming PC?
3. Based on the current situation, when can the customer realistically expect the order to arrive?
4. Given the repeated delays and changes in estimated delivery, what compensation options (if any) are available to the customer?
5. Is expedited shipping an option at this point, and if so, how would it affect the delivery timeline?
6. How can the urgency of this order be communicated and prioritized, considering the customer's upcoming gaming tournament next month?
7. What steps has customer service already taken to address this issue, and what additional actions can be taken to resolve it?
8. How can the customer receive more frequent and accurate updates about their order status going forward?"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.21s/it, est. speed input: 164.03 toks/s, output: 45.93 toks/s]


In [7]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. Each significant facet of the main query is represented in the sub-queries, ensuring a comprehensive exploration of the topic.

The sub-queries address the following key points from the main query:

1. Current status and reasons for delivery date changes
2. Specific factors causing delays
3. Realistic expected delivery time
4. Compensation options
5. Expedited shipping options
6. Prioritization of urgency
7. Actions taken by customer service
8. Future updates on order status

These sub-queries break down the main question into diverse dimensions, ensuring a thorough investigation of the subject matter. They cover the customer's concerns about delays, compensation, and the urgency of their order, as well as potential solutions like expedited shipping. Additionally, they address the need for better communication and updates from the company.

The sub-queries are well-structured and cover all major aspects of the customer's query, providing a comprehensive framework for addressing the issues raised. This thorough approach ensures that all relevant information is considered and that the customer's concerns are fully explored.

Overall, the sub-queries demonstrate excellent breadth and depth, effectively covering all major aspects of the main query and providing a solid foundation for a comprehensive investigation of the topic.

__Score:__
3

In [8]:
# Okish decomposition

sub_queries = """1. What is the current status of order #AC-7892?
Why has the delivery date changed multiple times?
When will the custom-built gaming PC actually arrive?
What are your store hours on weekends?
How can I get better customer service support?
8. Can the order be expedited?
9. What is the name of the person that is responsible for the company?
"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = judge.evaluate(eval_input)
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.10s/it, est. speed input: 130.86 toks/s, output: 43.09 toks/s]


__Feedback:__
The generated sub-queries demonstrate a good attempt to cover various aspects of the main query, but they fall short of achieving excellent breadth. 

The sub-queries address several key points from the main query:
1. The current status of the order (sub-query 1)
2. The reason for multiple delivery date changes (sub-query 2)
3. The expected arrival time of the order (sub-query 3)
4. The possibility of expediting the order (sub-query 8)

These sub-queries effectively cover the main concerns about order status, delivery delays, and potential solutions.

However, there are noticeable gaps in coverage:
- The sub-queries don't address the customer's experience with customer service, which was a significant part of the original query.
- There's no sub-query specifically about compensation for the repeated delays, which was another key concern in the main query.
- The sub-queries don't address the urgency of the situation due to the upcoming gaming tournament, which was mentioned in the main query.

While the sub-queries do a good job of breaking down the main query into several dimensions, they miss some important facets. Answering these sub-queries would provide a partial exploration of the main topic, but not a comprehensive one.

Therefore, based on the evaluation criteria and scoring rubric, the sub-queries should be rated as:

<score>
2
</score>

__Score:__
2